## Deep Learning Approaches for RF-based detection & classification

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random

# import the torch packages
from torch.nn import Module
from torch.nn import Conv2d
from torch.nn import Linear
from torch.nn import MaxPool2d
from torch.nn import ReLU
from torch.nn import LogSoftmax
from torch import flatten
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset

import torchvision.models as models

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

# import custom functions
from helper_functions import *
from latency_helpers import *
from loading_functions import *

from sklearn import preprocessing

### Load Features

In [5]:
feat_folder = '../Features/'
feat_name = 'SPEC'
seg_len = 20
# datestr = '2022-07-05'
n_per_seg = 256
interferences = ['CLEAN']
dataset = load_dronedetect_data(feat_folder, feat_name, seg_len, n_per_seg, interferences)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 80/80 [02:08<00:00,  1.60s/it]


In [6]:
print('dataset size', len(dataset))
print('shape of each item', dataset.__getitem__(10)[0].shape)

dataset size 9487
shape of each item torch.Size([129, 4687])


## Transfer learning from Resnet50 & Apply Logistic Regression (Swinney paper)

In [19]:
# use pretrained resnet feature and just keep up to the last layer
# resnet50 = models.resnet50(pretrained=True)
# modules=list(resnet50.children())[:-1]
# # resnet50=nn.Sequential(*modules)
# for p in resnet50.parameters():
#     p.requires_grad = False

In [20]:
# test resnet
# input = torch.randn(1,1,30,300)
# inputr = input.repeat(1,3,1,1)
# resnet50(inputr).shape

torch.Size([1, 1000])

In [21]:
# resnet_feats = []
# resnet_y = []
# for n in range(len(dataset)):
#     d = dataset.__getitem__(n)
#     inarr = d[0]
#     inputr = inarr.repeat(1,3,1,1)  # repeat to have 3 channels of the same info
#     out = resnet50(inputr)
#     resnet_feats.append(np.array(out))
#     resnet_y.append(np.array(d[1]))

# resnet_feats = np.array(resnet_feats)
# resnet_y = np.array(resnet_y)

# # flatten the middle dimension
# resnet_feats = resnet_feats.reshape(resnet_feats.shape[0], resnet_feats.shape[-1])
# # invert labels back to categorical
# resnet_y_cat = dataset.le.inverse_transform(resnet_y.astype(np.int64))

## Transfer learning VGG16

In [11]:
vgg16 = models.vgg16()
# summary(vgg16, (3,224,224))

modules=list(vgg16.children())[:-1]
vggfeats=nn.Sequential(*modules)

for p in vggfeats.parameters():
    p.requires_grad = False

In [12]:
# test one input
d = dataset.__getitem__(0)
inarr = d[0]
inputr = inarr.repeat(1,3,1,1)
# inputr = inputr.to(device)
out = vggfeats(inputr)

# reshape the output
out.flatten()

tensor([4.0510e-10, 5.0208e-10, 2.3334e-09,  ..., 8.1137e-12, 2.3012e-10,
        5.5576e-10])

In [ ]:
vgg_feats = []
vgg_y = []
for n in tqdm(range(len(dataset))):
    d = dataset.__getitem__(n)
    inarr = d[0]
    inputr = inarr.repeat(1,3,1,1)  # repeat to have 3 channels of the same info
    out = vggfeats(inputr)
    vgg_feats.append(np.array(out.flatten()))
    vgg_y.append(np.array(d[1]))

vgg_feats = np.array(vgg_feats)
vgg_y = np.array(vgg_y)

# flatten the middle dimension
vgg_feats = vgg_feats.reshape(vgg_feats.shape[0], vgg_feats.shape[-1])
# invert labels back to categorical
vgg_y_cat = dataset.le.inverse_transform(vgg_y.astype(np.int64))

### TO DO: Save these features to be easily loaded [ this took almost 2 hours]

 48%|█████████████████████████████████████████████████████████▉                                                              | 4580/9487 [53:35<55:49,  1.47it/s]

In [49]:
feats_lr = vgg_feats # which features to use for logit reg
y_cat = vgg_y_cat

In [62]:
# split data into K-fold
k_fold = 10
cv = KFold(n_splits=k_fold, random_state=1, shuffle=True)

# model parameters
Cs=list(map(lambda x:pow(2,x),range(-2,10,5)))

best_params_ls = []
acc_ls = []
f1_ls = []
runt_ls = []

parameters = {'C':Cs}

for train_ix, test_ix in tqdm(cv.split(feats_lr)):
    
    # find the optimal hypber parameters
    lr = LogisticRegression(max_iter=1000)
    clf = GridSearchCV(lr, parameters, n_jobs=1)
    clf.fit(feats_lr[train_ix], y_cat[train_ix])
    
    print(clf.best_params_)
    best_params_ls.append(clf.best_params_)
    
    # predict on the test data
    y_pred, runtimes = atomic_benchmark_estimator(clf, feats_lr[test_ix], output_type= '<U3', 
                                                  verbose=False)
    runt_ls.append(np.mean(runtimes))
    
    acc = accuracy_score(y_cat[test_ix], y_pred)
    f1 = f1_score(y_cat[test_ix], y_pred, average='weighted')
    print('Accuracy: {:.3},\t F1: {:.3}'.format(acc,f1))
    acc_ls.append(acc)
    f1_ls.append(f1)
    
out_msg = feat_name+': ResNet+LR average test acc: {:.2}, F1: {:.2}, Run-time: {:.2}ms'.format(np.mean(acc_ls), np.mean(f1_ls), np.mean(runt_ls)*1e3)
print(out_msg)

0it [00:00, ?it/s]/home/kzhou/.pyenv/versions/3.8.2/envs/main/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/kzhou/.pyenv/versions/3.8.2/envs/main/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://sci

{'C': 8}
Accuracy: 0.155,	 F1: 0.153


/home/kzhou/.pyenv/versions/3.8.2/envs/main/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/kzhou/.pyenv/versions/3.8.2/envs/main/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stab

{'C': 256}
Accuracy: 0.13,	 F1: 0.128


/home/kzhou/.pyenv/versions/3.8.2/envs/main/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/kzhou/.pyenv/versions/3.8.2/envs/main/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stab

...

{'C': 0.25}
Accuracy: 0.15,	 F1: 0.144


/home/kzhou/.pyenv/versions/3.8.2/envs/main/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/kzhou/.pyenv/versions/3.8.2/envs/main/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stab

{'C': 0.25}
Accuracy: 0.16,	 F1: 0.159


/home/kzhou/.pyenv/versions/3.8.2/envs/main/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
7it [54:29, 467.06s/it]


KeyboardInterrupt: 

In [55]:
acc = accuracy_score(y_cat[test_ix], y_pred)
f1 = f1_score(y_cat[test_ix], y_pred, average='weighted')
print('Accuracy: {:.3},\t F1: {:.3}'.format(acc,f1))

Accuracy: 0.141,	 F1: 0.139


In [61]:
Cs

[0.25, 1, 4, 16, 64, 256]

## 3. Apply resnet & a fully connected layer

In [ ]:
# # Define network
# model = torch.hub.load('pytorch/vision:v0.6.0', 'vgg19', pretrained=True)
# model.classifier = nn.Linear(model.classifier[0].in_features, num_classes)
# print(model)

In [33]:
class ResnetFC(nn.Module):
    def __init__(self, num_classes):
        super(ResnetFC,self).__init__()
        self.num_classes = num_classes
        self.resnetfull = models.resnet50(pretrained=True)
        modules=list(vgg16.children())[:-1]
        self.resnetfeats=nn.Sequential(*modules)
        
        for param in self.resnetfeats.parameters():
            self.resnetfeats.requires_grad_(False)
        
        self._fc = nn.Linear(1000, num_classes)
    def forward(self, x):
#         batch_size ,_,_ =x.shape
        
        # replicate the image to have 3 channels
        x = x.repeat(1,3,1,1)
        x = self.resnetfeats(x)
        x = self._fc(x)
        
        return x

In [34]:
fctest = ResnetFC(7)

In [35]:
fctest.forward(dataset.__getitem__(10)[0])

tensor([[-0.0363, -1.0948,  0.0186,  0.1136, -0.1828,  0.1280, -0.2705]],
       grad_fn=<AddmmBackward0>)

### Kfold CV

In [ ]:
from nn_functions import runkfoldcv

In [ ]:
# Configuration options
k_folds = 2

batch_size = 8 # 128
num_classes = 7
learning_rate = 0.01
num_epochs = 1 # 0
momentum = 0.95
l2reg = 1e-4

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [46]:
runkfoldcv(fctest, dataset, device, k_folds, batch_size, learning_rate, num_epochs, momentum, l2reg)

--------------------------------
FOLD 0
--------------------------------
Starting epoch 1


RuntimeError: CUDA out of memory. Tried to allocate 4.65 GiB (GPU 0; 23.65 GiB total capacity; 11.67 GiB already allocated; 2.77 GiB free; 11.69 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF